# Model 3 - UDE Framework (SIR Epidemic Model)

**System:**
$$\frac{dS}{dt} = -\beta(t) \cdot S$$
$$\frac{dI}{dt} = \beta(t) \cdot S - \gamma I$$
$$\frac{dR}{dt} = \gamma I$$

**Unknown:** Function $\beta(t)$ (infection rate as function of time)  
**Unknown Parameter:** $\gamma$ (recovery rate)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn

from ude_framework import (
    DataGenerator, 
    create_ode_system,
    create_neural_network,
    create_ude,
    UDETrainer,
    UDEEvaluator
)

%matplotlib inline
np.random.seed(42)
torch.manual_seed(42)

print("✓ UDE Framework loaded!")


In [ ]:
# ============================================================================
# CONFIGURE EVERYTHING HERE
# ============================================================================

# ----------------------------------------------------------------------------
# 1. TRUE ODE SYSTEM (for generating data)
# ----------------------------------------------------------------------------
def true_ode_equations(t, y, params):
    """
    Model 3 TRUE system (SIR model):
    dS/dt = -β(t)*S
    dI/dt = β(t)*S - γ*I
    dR/dt = γ*I
    
    Here we assume β(t) varies with time (e.g., time-dependent contact rate)
    """
    S, I, R = y
    beta_0, beta_amp, omega, gamma = params['beta_0'], params['beta_amp'], params['omega'], params['gamma']
    
    # True β(t): sinusoidal variation (e.g., seasonal effects)
    beta_t = beta_0 + beta_amp * np.sin(omega * t)
    
    dS_dt = -beta_t * S * I
    dI_dt = beta_t * S * I - gamma * I
    dR_dt = gamma * I
    
    return [dS_dt, dI_dt, dR_dt]

# True system parameters
TRUE_PARAMS = {
    'beta_0': 0.5,      # baseline infection rate
    'beta_amp': 0.2,    # amplitude of seasonal variation
    'omega': 0.3,       # frequency of variation
    'gamma': 0.1        # recovery rate (UNKNOWN in UDE)
}

# State variable names
STATE_NAMES = ['S', 'I', 'R']

# Initial conditions (normalized population)
INITIAL_CONDITIONS = np.array([0.99, 0.01, 0.0])

# Time settings
T_START = 0.0
T_END = 50.0
N_POINTS = 1000

# Noise settings
NOISE_LEVEL = 0.03
NOISE_TYPE = 'relative'

# ----------------------------------------------------------------------------
# 2. UDE STRUCTURE
# ----------------------------------------------------------------------------
def ude_ode_equations(t, y, nn_outputs, known_params):
    """
    UDE for Model 3:
    - KNOWN: SIR structure
    - UNKNOWN: β(t) learned by NN (as function of time and state)
    - LEARNABLE PARAMETER: γ
    """
    S = y[..., 0:1]
    I = y[..., 1:2]
    R = y[..., 2:3]
    
    # Get NN output for β(t)
    beta_t_nn = nn_outputs['beta_nn']
    if beta_t_nn.dim() > 2:
        beta_t_nn = beta_t_nn.squeeze(-1)
    
    gamma = known_params.get('gamma_learnable', torch.tensor(0.1))
    
    # SIR dynamics with learned β(t)
    dS_dt = -beta_t_nn * S * I
    dI_dt = beta_t_nn * S * I - gamma * I
    dR_dt = gamma * I
    
    return torch.cat([dS_dt, dI_dt, dR_dt], dim=-1)

KNOWN_PARAMS = {}

# ----------------------------------------------------------------------------
# 3. NEURAL NETWORK CONFIGURATION
# ----------------------------------------------------------------------------
NN_INPUT_DIM = 3   # β depends on time and states (t, S, I)
NN_OUTPUT_DIM = 1  # outputs β(t)

def nn_input_extractor(y):
    """Extract states for β(t) - use all three states"""
    return y  # Use full state vector

# Architecture
NN_ARCHITECTURE = 'flexible'
NN_CONFIG = {
    'hidden_dims': [64, 64, 64],
    'activation': 'tanh',
    'final_activation': 'softplus',  # β should be positive
    'use_batch_norm': False,
    'dropout': 0.0
}

NN_NAME = 'beta_nn'

# ----------------------------------------------------------------------------
# 4. TRAINING CONFIGURATION
# ----------------------------------------------------------------------------
N_EPOCHS = 500
LEARNING_RATE = 1e-3
OPTIMIZER = 'adam'
WEIGHT_DECAY = 0.0
GRAD_CLIP = 1.0

SCHEDULER_TYPE = 'plateau'
SCHEDULER_PARAMS = {
    'factor': 0.5,
    'patience': 80,
    'min_lr': 1e-6,
    'verbose': False
}

ODE_SOLVER = 'dopri5'
ODE_RTOL = 1e-6
ODE_ATOL = 1e-8

LOSS_TYPE = 'mse'
STATE_WEIGHTS = None
PRINT_EVERY = 20

# ----------------------------------------------------------------------------
# 5. OPTIONAL: TRUE FUNCTION
# ----------------------------------------------------------------------------
def true_function_for_comparison(nn_input):
    """True β(t) - time-dependent"""
    # For visualization, assume nn_input is time values
    t = nn_input
    beta_0, beta_amp, omega = TRUE_PARAMS['beta_0'], TRUE_PARAMS['beta_amp'], TRUE_PARAMS['omega']
    return beta_0 + beta_amp * np.sin(omega * t)

FUNCTION_INPUT_RANGE = np.linspace(0, 50, 300)
FUNCTION_INPUT_NAME = 'Time'
FUNCTION_OUTPUT_NAME = 'β(t)'

print("✓ Configuration complete!")


## 1. Generate Data


In [ ]:
true_system = create_ode_system(
    name="Model 3 True System (SIR)",
    equations=true_ode_equations,
    params=TRUE_PARAMS,
    state_names=STATE_NAMES
)

data_gen = DataGenerator(true_system)
data = data_gen.generate(
    initial_conditions=INITIAL_CONDITIONS,
    t_span=(T_START, T_END),
    n_points=N_POINTS,
    noise_level=NOISE_LEVEL,
    noise_type=NOISE_TYPE,
    random_seed=42
)

t = data['t']
y_true = data['y_true']
y_noisy = data['y_noisy']
y0 = data['y0']

print(f"✓ Generated {len(t)} points from t={T_START} to {T_END}")
print(f"  States: {STATE_NAMES}")
print(f"  Noise: {NOISE_LEVEL*100}% {NOISE_TYPE}")


In [ ]:
n_states = len(STATE_NAMES)
fig, axes = plt.subplots(n_states, 1, figsize=(12, 3*n_states))

for i, (ax, name) in enumerate(zip(axes, STATE_NAMES)):
    ax.plot(t, y_true[:, i], 'b-', label=f'True', linewidth=2)
    ax.plot(t, y_noisy[:, i], 'r.', label=f'Noisy', alpha=0.5, markersize=2)
    ax.set_ylabel(name, fontsize=12)
    ax.legend()
    ax.grid(True, alpha=0.3)

axes[-1].set_xlabel('Time', fontsize=12)
plt.suptitle('Model 3: SIR Epidemic Data', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()


## 2. Build UDE Model


In [ ]:
nn_model = create_neural_network(
    input_dim=NN_INPUT_DIM,
    output_dim=NN_OUTPUT_DIM,
    architecture=NN_ARCHITECTURE,
    **NN_CONFIG
)

print("Neural Network:")
print(nn_model)
print(f"\nParameters: {nn_model.count_parameters():,}")


In [ ]:
class NNWrapper(nn.Module):
    def __init__(self, nn, input_extractor):
        super().__init__()
        self.nn = nn
        self.input_extractor = input_extractor
    
    def forward(self, t, y):
        nn_input = self.input_extractor(y)
        return self.nn(t, nn_input)

wrapped_nn = NNWrapper(nn_model, nn_input_extractor)

gamma_learnable = nn.Parameter(torch.tensor(0.15))  # Initial guess
KNOWN_PARAMS['gamma_learnable'] = gamma_learnable

ude_model = create_ude(
    n_states=len(STATE_NAMES),
    ode_func=ude_ode_equations,
    neural_networks={NN_NAME: wrapped_nn},
    known_params=KNOWN_PARAMS
)

ude_model.register_parameter('gamma_learnable', gamma_learnable)

print(f"\n✓ UDE Model created!")
print(f"  NN parameters: {ude_model.count_parameters()['total']}")
print(f"  Initial γ: {gamma_learnable.item():.4f} (True: {TRUE_PARAMS['gamma']})")


## 3. Train UDE


In [ ]:
t_torch = torch.tensor(t, dtype=torch.float32)
y_noisy_torch = torch.tensor(y_noisy, dtype=torch.float32)
y_true_torch = torch.tensor(y_true, dtype=torch.float32)
y0_torch = torch.tensor(y0, dtype=torch.float32)

trainer = UDETrainer(
    ude_model=ude_model,
    optimizer_name=OPTIMIZER,
    learning_rate=LEARNING_RATE,
    scheduler_type=SCHEDULER_TYPE,
    scheduler_params=SCHEDULER_PARAMS,
    weight_decay=WEIGHT_DECAY,
    grad_clip=GRAD_CLIP,
    ode_solver=ODE_SOLVER,
    ode_rtol=ODE_RTOL,
    ode_atol=ODE_ATOL
)

print("✓ Trainer initialized")


In [ ]:
print(f"\n{'='*60}")
print(f"TRAINING ({N_EPOCHS} epochs)")
print(f"{'='*60}\n")

trainer.train(
    y0=y0_torch,
    t=t_torch,
    y_true=y_noisy_torch,
    n_epochs=N_EPOCHS,
    weights=STATE_WEIGHTS,
    loss_type=LOSS_TYPE,
    print_every=PRINT_EVERY
)

print(f"\nLearned γ: {ude_model.gamma_learnable.item():.4f} (True: {TRUE_PARAMS['gamma']})")


In [ ]:
history = trainer.get_history()
evaluator = UDEEvaluator(ude_model, STATE_NAMES)
evaluator.plot_training_history(history)


## 4. Evaluate UDE Performance


In [ ]:
with torch.no_grad():
    y_pred = trainer.forward_solve(y0_torch, t_torch).numpy()

metrics = evaluator.compute_metrics(y_pred, y_true)
evaluator.print_metrics(metrics)


In [ ]:
evaluator.plot_trajectories(
    t=t,
    y_pred=y_pred,
    y_true=y_true,
    y_noisy=y_noisy
)


## 5. Visualize Learned β(t)


In [ ]:
# Extract β(t) along the trajectory
with torch.no_grad():
    y_pred_torch = torch.tensor(y_pred, dtype=torch.float32)
    beta_learned = []
    for i in range(len(t)):
        beta_t = wrapped_nn(t_torch[i:i+1], y_pred_torch[i:i+1])
        beta_learned.append(beta_t.item())
    beta_learned = np.array(beta_learned)

# True β(t)
beta_true = true_function_for_comparison(t)

# Plot
plt.figure(figsize=(12, 5))
plt.plot(t, beta_true, 'b-', linewidth=2, label='True β(t)')
plt.plot(t, beta_learned, 'r--', linewidth=2, label='Learned β(t)')
plt.xlabel('Time', fontsize=12)
plt.ylabel('β(t)', fontsize=12)
plt.title('Infection Rate Function', fontsize=14, fontweight='bold')
plt.legend(fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

print(f"β(t) R² score: {1 - np.sum((beta_learned - beta_true)**2) / np.sum((beta_true - np.mean(beta_true))**2):.4f}")


## 6. Parameter Comparison
10200

In [ ]:
print("\n" + "="*60)
print("PARAMETER COMPARISON")
print("="*60)
print(f"\nParameter γ (recovery rate):")
print(f"  True value:    {TRUE_PARAMS['gamma']:.4f}")
print(f"  Learned value: {ude_model.gamma_learnable.item():.4f}")
print(f"  Error:         {abs(ude_model.gamma_learnable.item() - TRUE_PARAMS['gamma']):.4f}")
print(f"  Relative error: {abs(ude_model.gamma_learnable.item() - TRUE_PARAMS['gamma'])/TRUE_PARAMS['gamma']*100:.2f}%")
print("="*60)
